2. Zadatak: Statističke analize uzorka podataka
- Za svaku kategoričku varijablu prikazati distribucija frekvencije u grafu 
- Za svaku kontinuiranu varijablu izračunati srednju vrijednost, z vrijednosti, zaobljenost distribucije, nagnutost distribucije
- Izračunati korelacije svake varijable sa ciljnom varijablom
- Izračunati korelacije svake varijable izražene kroz Z vrijednost i  kategorizirane u razrede <-3 , (-3 do 3),>3 s ciljnom varijablom
- Objasniti što ste spoznali vezano uz promatrani skup podataka  temeljem analiza iz točke 2
- Objasniti / komentirati nalaze
- Sve mora biti vidljivo u Jupyter Notebooku, kod i rezultati,  kao i komentari


---

## 1. Učitavanje potrebnih biblioteka i učitavanje podataka

Prvo ćemo učitati sve potrebne biblioteke za analizu podataka:

In [1]:
# Učitavanje potrebnih biblioteka
import pandas as pd         
import numpy as np           

# Prikaži verzije biblioteka
print("Verzije biblioteka:")
print(f"Pandas: {pd.__version__}")
print(f"NumPy: {np.__version__}")

# Učitavanje podataka iz CSV datoteke iz dataset mape
# read_csv je standardna Pandas funkcija za učitavanje podataka iz CSV formata
df_original = pd.read_csv('../dataset/student-por.csv', sep=';')

# Kreiramo radnu kopiju s kojom ćemo raditi (backup originala u memoriji)
df = df_original.copy()

print("Datoteka uspješno učitana!")
print(f"\nDimenzije datoteke: {df.shape}")
print(f"  - Broj redaka (zapisa): {df.shape[0]}")
print(f"  - Broj stupaca (atributa): {df.shape[1]}")

Verzije biblioteka:
Pandas: 2.3.3
NumPy: 2.4.0
Datoteka uspješno učitana!

Dimenzije datoteke: (649, 33)
  - Broj redaka (zapisa): 649
  - Broj stupaca (atributa): 33
